Tuwaiq Academy, SDAIA, ALLAM-Challenge, 2024

***Abdulrahman Alshahrani, Abdullah Alwakeel***

# Introduction
This notebook is our implementation of our idea in the Allam-Challenge. It is essentially a structured approach to using a Language Model (LLM) to "force" the LLM to generate poetry in any of the many poetry types (bohours). Through this notebook, we aim to explore and demonstrate the capabilities of LLMs in generating structured poetic forms, adhering to the rules and styles of various traditional poetry types.

# Explination

This program is designed to generate Arabic poetry, specifically focusing on the correct structure and rules of Arabic poetry, such as maintaining correct Wazn (rhythm) and Qafiya (rhyme). It utilizes machine learning models, validation mechanisms, and feedback loops to ensure the generated poetry adheres to linguistic and stylistic constraints.

## Key Components
### **Language Model (LLM)**: 
At the core, an LLM is used to generate the actual lines of poetry based on a prompt. This model generates text while considering provided context and constraints.

### **RAG (Retrieval Augmented Generation)**:
This module wraps user prompts with context, such as examples of words that fit the required Qafiya (rhyme) and previous successful lines. It guides the LLM to generate poetry that follows the intended form.

### **Extractor**:
Automatically extracts Qafiya and Wazn from the generated line to validate them against the given constraints.

### **Validators**:
#### 1. **WaznValidator**:
Ensures that the generated line conforms to the correct Wazn pattern.
#### 2. **QafiyaValidator**:
Ensures that the Qafiya (rhyme) of the generated line matches the expected one.

### **Feedback Generator**:
Provides feedback when the generated line doesn't produce the expected Qafiya or Wazn, which is sent back to the LLM to produce better lines.

### **ShatrGenerator**:
This is the central class responsible for orchestrating the poetry generation process. It interacts with the LLM, RAG, validators, and feedback generator to iteratively produce lines that meet the Wazn and Qafiya requirements.

## **Workflow**:
1. The user inputs a prompt.
2. The program generates a line (called "shatrs") using the LLM.
3. It extracts and validates the Wazn (rhythm) and Qafiya (rhyme) of the generated line.
4. If the line is invalid, a feedback is generated, and we go back to step 2 with the new feedback.
5. if the line is valid, we save the line and repeat the process until we create enough lines for a complete poem.

### Flow Diagram

<!-- ![Flow Diagram](flow-diagram.svg) -->
<img src="flow-diagram.svg" alt="Flow Diagram" width="800">


# Implementation

## Requirements

In [119]:
%pip install ibm_cloud_sdk_core

Note: you may need to restart the kernel to use updated packages.


## LLM Interface
An interface class for interacting with large language models. Any LLM implementation must define the `generate` method, which takes a prompt and generates text based on it.

In [79]:
from abc import ABC, abstractmethod
class LLM_Interface(ABC):
    @abstractmethod
    def generate(self, prompt, **kwargs) -> str:
        pass

This is the `ALLAM` class which implements the `LLM_Interface` abstract class

In [118]:
import requests
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

BASE_URL = "https://eu-de.ml.cloud.ibm.com/ml/"

class ALLAM(LLM_Interface):
    def __init__(self, API_KEY):
        self.model_id = "sdaia/allam-1-13b-instruct"
        self.project_id = "0a443bde-e9c6-41dc-b1f2-65c6292030e4"

        # get authentication token
        authenticator = IAMAuthenticator(API_KEY)
        token = authenticator.token_manager.get_token()
        self.headers = {
            'Accept': 'application/json',
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {token}'
        }

        # set default parameters
        self.parameters = {
            "decoding_method": "sample",
            "max_new_tokens": 30,
            "temperature": 1,
            "top_k": 50,
            "top_p": 1,
            "repetition_penalty": 2,
        }

    def generate(self, prompt, **kwargs):
        url = BASE_URL + "v1/text/generation?version=2024-08-30"
        self.body = {
            "input": f"<s> [INST] {prompt} [/INST]",
            "model_id": self.model_id,
            "project_id": self.project_id,
            "parameters": self.parameters
        }

        response = requests.post(url, headers=self.headers, json=self.body)
        response.raise_for_status()
        
        data = response.json()
        return data['results'][0]['generated_text']

## RAG (Retrieval Augmented Generation)

The RAG class provides context to the LLM by wrapping user input with example rhymes (Qafiya) and previous successful lines. It reads a database of words and processes them to suggest suitable rhyme words.

In [100]:
import json
import random

class RAG:
    def __init__(self, filepath="qawafi-database.json", qafiya=None):
        self.qafiya = qafiya
        self.message = None
        self.db = None
        with open(filepath, 'r', encoding="utf-8") as f:
            self.db = json.load(f)["data"]
        self.setQafiya(qafiya)
    
    def wrap(self, prompt, previous_shatrs=None, feedback=None):
        full_text = ""
        full_text += "اكتب شطر واحد لجزء من قصيدة.\n"
        
        if self.qafiya:
            full_text += f" قافية القصيدة هي '{self.qafiya}'. "

        if self.message:
            full_text += "هنا بعض الامثلة لكلمات تنتهي بهذه القافية: \n"
            # full_text += "<QafiyaExamples>"
            full_text += ", ".join(random.sample(self.message, 10))
            full_text += "\n"
            # full_text += "</QafiyaExamples>"
        
        if prompt:
            full_text += f"هنا الطلب اللي وضعه المستخدم:\n"
            full_text += f"{prompt}\n"
        
        if previous_shatrs:
            full_text += "هنا الشطور السابقة:\n"
            full_text += f"{previous_shatrs}\n"

        if feedback:
            full_text += "هنا بعض النصائح على هذا اخر شطر تم ادخاله:\n"
            full_text += f"{feedback}\n"

        return full_text
    
    def update(self, qafiya):
        self.message = self.setQafiya(qafiya)
        self.qafiya = qafiya

    def setQafiya(self, qafiya):
        if not qafiya:
            return None

        def processed(word):
            if word.endswith(qafiya):
                return word
            if qafiya[-1] == "ا" and word.endswith(qafiya[0]):
                return word+"ا"
            if qafiya[-1] == "ه" and word.endswith(qafiya[0]):
                return word+"ه"
            if qafiya[-1] == "ه" and word.endswith("ة"):
                return word.replace("ة" , f"ه")
            return None
        
        output = []
        for word in self.db:
            new_word = processed(word)
            if new_word:
                output.append(new_word)

        return output
        

#### Demoing some `RAG` examples

In [82]:
rag = RAG()

print(rag.wrap("اكتب قصيدة عن امرؤ القيس"))
rag.update("يا")
print(rag.wrap("اكتب قصيدة عن امرؤ القيس"))
rag.update("لا")
print(rag.wrap("اكتب قصيدة عن امرؤ القيس"))
print(rag.wrap("اكتب قصيدة عن امرؤ القيس", ["قَالَتْ فُطَيْمَةُ حَلِّ شِعْرَكَ مَدْحَهُ"]))
print(rag.wrap("اكتب قصيدة عن امرؤ القيس", ["قَالَتْ فُطَيْمَةُ حَلِّ شِعْرَكَ مَدْحَهُ", "أَفَبَعْدَ كِنْدَةَ تَمْدَحَنَّ قَبِيلا"], "الشطر السابق لا يحتوي على قافية"))

اكتب شطر واحد لجزء من قصيدة.
هنا الطلب اللي وضعه المستخدم:
اكتب قصيدة عن امرؤ القيس

اكتب شطر واحد لجزء من قصيدة.
 قافية القصيدة هي 'يا'. هنا بعض الامثلة لكلمات تنتهي بهذه القافية: 
عرايا, ينسيا, يغويا, يلفيا, مليشيا, قصايا, كوميديا, يبتغيا, شلايا, يدريا
هنا الطلب اللي وضعه المستخدم:
اكتب قصيدة عن امرؤ القيس

اكتب شطر واحد لجزء من قصيدة.
 قافية القصيدة هي 'لا'. هنا بعض الامثلة لكلمات تنتهي بهذه القافية: 
صلولا, علولا, تكفلا, زعبيلا, يتخيلا, آجالا, حوافلا, زلا, خاذلا, كولسترولا
هنا الطلب اللي وضعه المستخدم:
اكتب قصيدة عن امرؤ القيس

اكتب شطر واحد لجزء من قصيدة.
 قافية القصيدة هي 'لا'. هنا بعض الامثلة لكلمات تنتهي بهذه القافية: 
يوغلا, تواكلا, تأولا, مصاقلا, حثلا, تنصلا, يرتجلا, يزلزلا, يحنبلا, عبلا
هنا الطلب اللي وضعه المستخدم:
اكتب قصيدة عن امرؤ القيس
هنا الشطور السابقة:
['قَالَتْ فُطَيْمَةُ حَلِّ شِعْرَكَ مَدْحَهُ']

اكتب شطر واحد لجزء من قصيدة.
 قافية القصيدة هي 'لا'. هنا بعض الامثلة لكلمات تنتهي بهذه القافية: 
علا, يجلا, اجتمالا, ندلا, احتفالا, يرهلا, يختبلا, سائلا, يتحنجلا, ثعلا
هن

## Extractor
The Extractor class is responsible for extracting both the Qafiya and Wazn types from a generated line (shatr). This information is then used by the validators to ensure the correctness of the Qafiya and Wazn.

In [101]:
class Extractor:
    def __init__(self):
        pass

    def extract(self, shatr):
        qafiya_type = self.extract_qafiya(shatr)
        wazn_type = self.extract_wazn(shatr)
        return qafiya_type, wazn_type

    def extract_qafiya(self, shatr):
        # TODO
        return None

    def extract_wazn(self, shatr):
        # TODO
        return None

#### Demoing some `Extractor` examples

In [84]:
extractor = Extractor()

print(extractor.extract("قَالَتْ فُطَيْمَةُ حَلِّ شِعْرَكَ مَدْحَهُ"))
print(extractor.extract("أَفَبَعْدَ كِنْدَةَ تَمْدَحَنَّ قَبِيلا"))

(None, None)
(None, None)


## WaznValidator
This class is responsible for validating the Wazn of a given shatr. It compares the Wazn of the current shatr with the previous one.

In [102]:
class WaznValidator:
    def __init__(self):
        pass

    def validate_wazn(self, current_wazn, previous_wazn=None):
        # TODO: implement validation
        # if previous_wazn is None:
        #     return True
        # if current_wazn == previous_wazn:
        #     return True
        # else:
        #     return False
        return True

#### Demoing some `WaznValidator` examples

In [86]:
wazn_validator = WaznValidator()

print(wazn_validator.validate_wazn("طويل"))
print(wazn_validator.validate_wazn("طويل", "بسيط"))
print(wazn_validator.validate_wazn("طويل", "طويل"))

True
True
True


### QafiyaValidator
The QafiyaValidator class checks if the generated Qafiya matches the expected one. This ensures that the lines end with the correct sounds as required by the given Qafiya.

In [103]:
class QafiyaValidator:
    def __init__(self):
        pass

    def validate_qafiya(self, current_qafiya, previous_qafiya=None):
        # TODO: implement validation
        # if previous_qafiya is None:
        #     return True        
        # if current_qafiya == previous_qafiya:
        #     return True
        # else:
        #     return False
        return True

#### Demoing some `QafiyaValidator` examples

In [88]:
qafiya_validator = QafiyaValidator()

print(qafiya_validator.validate_qafiya("ه"))
print(qafiya_validator.validate_qafiya("ل", "ا"))
print(qafiya_validator.validate_qafiya("ل", "ل"))

True
True
True


### FeedbackGenerator

This class generates feedback for incorrect or invalid Wazn or Qafiya. It provides suggestions based on what went wrong to guide further iterations of line generation.

In [104]:
class FeedbackGenerator:
    def generate_feedback(self, type, invalid_item, expected_item, invalid_shatr,):
        if type == "qafiya":
            return self.qafiya_feedback(invalid_item, expected_item, invalid_shatr)
        
        if type == "wazn":
            return self.wazn_feedback(invalid_item, expected_item, invalid_shatr)
    
    def qafiya_feedback(self, invalid_qafiya, expected_qafiya, invalid_shatr):
        # TODO

        text = ""
        text += f"الشطر المدخل {invalid_shatr} غير صحيح. "
        text += f"القافية المدخلة {invalid_qafiya} غير صحيحة. "
        text += f"القافية الصحيحة هي {expected_qafiya}. "
        
        return text

    def wazn_feedback(self, invalid_wazn, expected_wazn, invalid_shatr):
        # TODO

        text = ""
        text += f"الشطر المدخل {invalid_shatr} غير صحيح. "
        text += f"الوزن المدخل {invalid_wazn} غير صحيح. "
        text += f"الوزن الصحيح هو {expected_wazn}. "

        return text

#### Demoing some `FeedbackGenerator` examples

In [92]:
generator = FeedbackGenerator()

print(generator.generate_feedback("qafiya", "ق", "ل", "أَفَبَعْدَ كِنْدَةَ تَمْدَحَنَّ قَبِيلا"))
print(generator.generate_feedback("wazn", "بسيط", "كامل", "قَالَتْ فُطَيْمَةُ حَلِّ شِعرَكَ مَدحَهُ"))

القافية المدخلة ق غير صحيحة. القافية الصحيحة هي ل. 
الوزن المدخل بسيط غير صحيح. الوزن الصحيح هو كامل. 


### ShatrGenerator

The `ShatrGenerator` class generates lines of poetry based on the given prompt. It uses a language model (LLM), a feedback mechanism, and validators for Wazn and Qafiya. The process involves:

- Generating a line (shatr) from the LLM class, which implements `LLM_Interface`.
- Extracting the Wazn and Qafiya from the `Extractor`
- Validating both the Wazn and Qafiya using `WaznValidator` and `QafiyaValidator`, respectively.
- Give feedback to the LLM for incorrect Wazn or Qafiya using `FeedbackGenerator`.
- Keep trying until it generates a correct shatr.

In [111]:
class ShatrGenerator:
    def __init__(self, llm, rag=None, feedback_generator=None, extractor=None, wazn_validator=None, qafiya_validator=None):
        self.llm = llm
        self.rag = rag or RAG()
        self.feedback_generator = feedback_generator or FeedbackGenerator()
        self.extractor = extractor or Extractor()
        self.wazn_validator = wazn_validator or WaznValidator()
        self.qafiya_validator = qafiya_validator or QafiyaValidator()
        
    def generate_shatr(self, prompt, wazn=None, qafiya=None, feedback=None, previous_shatrs=None):
        valid = False
        
        while not valid:
            # Step 1: Generate a shatr
            shatr = self.llm.generate(self.rag.wrap(prompt, previous_shatrs, feedback))
            print(shatr)

            # Step 2: Extract Wazn and Qafiya
            new_qafiya, new_wazn = self.extractor.extract(shatr)

            # Step 3: Validate Wazn
            valid_wazn = self.wazn_validator.validate_wazn(new_wazn, wazn)
            if not valid_wazn:
                feedback = self.feedback_generator.generate_feedback("wazn", shatr, wazn=new_wazn)
                continue  # Loop back to regenerate

            wazn = new_wazn

            # Step 5: Validate Qafiya
            valid_qafiya = self.qafiya_validator.validate_qafiya(new_qafiya, qafiya)
            if not valid_qafiya:
                feedback = self.feedback_generator.generate_feedback("qafiya", shatr, qafiya=new_qafiya)
                continue  # Loop back to regenerate

            # Step 6: Update RAG and finalize shatr
            self.rag.update(qafiya)
            valid = True
        return shatr


### generate_qasida (function)

This function generates a full qasida (poem) by producing individual lines of poetry using the `ShatrGenerator`.

In [115]:
import random

def infer_wazn(prompt):
    return None

def infer_qafiya(prompt):
    return None

def infer_length(prompt):
    return 6

def generate_qasida(prompt, shatr_generator):
    wazn = infer_wazn(prompt)
    qafiya = infer_qafiya(prompt)
    length = infer_length(prompt)
    
    shatrs = []
    for shatr_idx in range(length):
        shatr = shatr_generator.generate_shatr(prompt, wazn, qafiya, shatrs)
        shatrs.append(shatr)
    
    output = ""
    for i, shdr in enumerate(shatrs):
        output += shdr + ("\n" if i % 2 else " # ")
    
    return output

# Demonstration

Running the code

In [112]:
api_key = input("Enter API key: ")
llm = ALLAM(api_key)
shatr_generator = ShatrGenerator(llm)
while True:
    prompt = input("Enter a prompt: ")
    if not prompt or prompt == "exit":
        break
    qasida = generate_qasida(prompt, shatr_generator)
    print(qasida)

 على نتعرّف الجمال النّادرِ،
في ربوع الأوطان تتشابك الأساطير،
متاحف التاريخ تروي ألف قصّة وحكايةً، 
وتزرع جذور الإنتماء بكلّ حياةٍ وبلادة.
سنابل القمح ترنو كأم حانيةٍ،​أرض الكرامة والفخر تباهي بألوانها الساطعة.,​من رمال الصحاري إلى جبال تعانق السحاب ونعومة الشواطيء العريقة..,
وطني الغالي يُلهم القلوب ويلهِم
 على نتعرّف الجمال النّادرِ،
في ربوع الأوطان تتشابك الأساطير،
متاحف التاريخ تروي ألف قصّة وحكايةً، 
وتزرع جذور الإنتماء بكلّ حياةٍ وبلادة.
سنابل القمح ترنو كأم حانيةٍ،​أرض الكرامة والفخر تباهي بألوانها الساطعة.,​من رمال الصحاري إلى جبال تعانق السحاب ونعومة الشواطيء العريقة..,
وطني الغالي يُلهم القلوب ويلهِم
 في كلّ ركن منه تروى حكايات المجد والأمان*{ اخوانْ الرُّوح يَنْبُضون بهواهُ والنَّجْم يحميهُم بمناعة المكان }*
ترتقي الأماني وتكبر الأحلام بيننا والأرض تطبعنا بلمساتِ الوفاء والعطاء اللامتناهيين.*عشق وطنّا مزروع فينا كنبتة جميلة تستحوذ عنايتنا ونضال أبنائِه الشامخين*.  
مع كل سطر جديد نكتب معاً لنغمر القصيدة بمشاعر الفخر والولاء للوطن
 في كلّ ركن منه تروى حكايات المجد والأمان*{ ا

KeyboardInterrupt: Interrupted by user

#### Viewing the result

In [117]:
for shatr in qasida.split("\n"):
    print(shatr)

 على نتعرّف الجمال النّادرِ،
في ربوع الأوطان تتشابك الأساطير،
متاحف التاريخ تروي ألف قصّة وحكايةً، 
وتزرع جذور الإنتماء بكلّ حياةٍ وبلادة.
سنابل القمح ترنو كأم حانيةٍ،​أرض الكرامة والفخر تباهي بألوانها الساطعة.,​من رمال الصحاري إلى جبال تعانق السحاب ونعومة الشواطيء العريقة..,
وطني الغالي يُلهم القلوب ويلهِم #  في كلّ ركن منه تروى حكايات المجد والأمان*{ اخوانْ الرُّوح يَنْبُضون بهواهُ والنَّجْم يحميهُم بمناعة المكان }*
ترتقي الأماني وتكبر الأحلام بيننا والأرض تطبعنا بلمساتِ الوفاء والعطاء اللامتناهيين.*عشق وطنّا مزروع فينا كنبتة جميلة تستحوذ عنايتنا ونضال أبنائِه الشامخين*.  
مع كل سطر جديد نكتب معاً لنغمر القصيدة بمشاعر الفخر والولاء للوطن
 فِي مَوْطِن التّاريخ والحَضارة عفَّتُ،
شَموخ المَكَارِر بين القُلُوب تَرَفّعَتِ، 
مَع كُل خَطوَة يَلتحم الوَلَاء وتُزدهر الشُعورَات,
رَوْنَق التّض الحيآة إلَى عُمُق الذّزاكِير بِفَخْر سَيّارَاتِ* اِبنُون الجَبَال تُعَانِق السّحاب وَتَكّلل الْمَح #  فى مواكب النصر تكلل الهامات وترتقيّ،*(على تراب العز تُغرس الأرواح وتنتشي*)فوطنّي ميلاد الإصرار وطالع ا